## Als erstes die Framewerks importieren, die Daten laden und als Pandas DataFrames verwenden

In [ ]:
import pandas as pd # für Datenimport
import numpy as np
import os # für Dateipfade
import sys
from datetime import datetime # um Datum und Uhrzeit im Überblick auszugeben



# Dateinamen definieren und Dateipfade erstellen 
customers_file = "data/customers.csv"
customers_path = os.path.join(os.getcwd(), customers_file)
offers_file = "data/offers.csv"
offers_path = os.path.join(os.getcwd(), offers_file)
contacts_file = "data/contacts.csv"
contacts_path = os.path.join(os.getcwd(), contacts_file)

# Datensätze als Pandas-Dataframes laden 
customers_data = pd.read_csv(customers_path)
offers_data = pd.read_csv(offers_path)
contacts_data = pd.read_csv(contacts_path)

## DataFrames aufräumen

### Datensatz Contacts

In [ ]:
# Die ersten beiden Spalten von contacts_data streichen
# musste erste Spalte der CSV anpassen um greifen zu können
contacts_data = contacts_data.drop(['Unnamed', 'Unnamed: 0'], axis=1) 

# contacts_data.index = contacts_data["person"] # person zum Index machen

### Datensatz Customers

In [ ]:
# For-Schleife zum Vergleichen der Werte in den ersten beiden Spalten
# Können die ersten beiden Spalten gestrichen werden?
for index, row in customers_data.iterrows():
    if row['Unnamed: 0'] - row['Unnamed: 0.1'] != 0:
        print(f"Fehler in Zeile {index+1}: Wert von Spalte B ist ungleich Spalte A")

# Entscheidung dafür, die ersten beiden Spalten von customers_data zu streichen
# drop-Funktion gibt es ein neues DataFrame aus
customers_data = customers_data.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1) 

# customers_data.index = customers_data["cust_id"] # ID zum Index machen

# Datumsangaben in customers.csv zu einem datetime64 Object aus Pandas umwandeln, 
# um danach weitere Berechnungen machen zu können
customers_data["became_member_on"] = pd.to_datetime(customers_data["became_member_on"], format='%Y%m%d')

In [ ]:
# Laut Frau Anderl ist in der Spalte "age" 118 lediglich ein Platzhalter für einen leeren Wert
# Um Durchschnitt, Median und co. nicht zu verfälschen: Umwandeln zu NaN
customers_data.loc[customers_data['age'] == 118, 'age'] = np.nan

In [ ]:
customers_data[(customers_data["age"] == 118)]

### Datensatz Offers

In [ ]:
offers_data = offers_data.drop(["Unnamed"], axis=1)

## Überblick über die Datensätze

### Contacts

In [ ]:
contacts_data.info()
contacts_data.head()

In [ ]:
contacts_data['event'].unique()

In [ ]:
contacts_data[~contacts_data.event.str.contains("transaction")]["event"].unique()

In [ ]:
# Gib mir alle Zeilen aus, die NICHT "transaction" enthalten und zähle dann, wie oft welcher
# Wert in der Spalte "val" vorkommt
contacts_data[~contacts_data.event.str.contains("transaction")]["val"].value_counts()

In [ ]:
received_data = contacts_data[contacts_data.event.str.contains("offer received")]["val"].value_counts()
offers_data["received"] = offers_data["id"].map(received_data)

In [ ]:
viewed_data = contacts_data[contacts_data.event.str.contains("offer viewed")]["val"].value_counts()
offers_data["viewed"] = offers_data["id"].map(viewed_data)

In [ ]:
completed_data = contacts_data[contacts_data.event.str.contains("offer completed")]["val"].value_counts()
offers_data["completed"] = offers_data["id"].map(completed_data)

In [ ]:
offers_data

### Customers

In [ ]:
customers_data.info()
customers_data.head()

### Offers

In [ ]:
offers_data.info()
offers_data.head()

## Auswertung

Hier schauen wir uns wir uns an, wer wie oft kontaktiert wurde. 
- Wie ist der Durchschnitt?
- Wer sind die 10 Personen, die am häufigsten konktakiert wurden?

Weitergehend möglich: 
- Welche Aussage können wir über die top 10 Personen treffen?
- Welche Personen wurden noch nicht kontaktiert?

In [ ]:
contacts_per_person = contacts_data['person'].value_counts()
contacts_per_person

In [ ]:
customers_data["number of contacts"] = customers_data["cust_id"].map(contacts_per_person)
customers_data.head()

In [ ]:
customers_data["number of contacts"].describe()

In [ ]:
customers_data[customers_data["number of contacts"].isnull()]

Wir wollten uns jetzt anschauen, wie lange es dauert, bis Angebote eingelöst werden.

In [ ]:
contacts_data.loc[contacts_data["event"] == "offer received"]

In [ ]:
# Filtere den contacts_data DataFrame nur für "offer received" und "offer complete" Ereignisse
offer_received = contacts_data.loc[contacts_data["event"] == "offer received"]
offer_completed = contacts_data.loc[contacts_data["event"] == "offer complete"]

# Verbinde (merge) die DataFrames basierend auf der Spalte "person"
merged_data = pd.merge(offer_received, offer_completed, on="person", suffixes=("_received", "_completed"))

# Berechne die Dauer zwischen "time_received" und "time_completed"
merged_data["duration"] = merged_data["time_completed"] - merged_data["time_received"]

# Füge die berechnete Dauer in das ursprüngliche contacts_data DataFrame ein
contacts_data = pd.merge(contacts_data, merged_data[["person", "duration"]], on="person", how="left")

# Aktualisiere die "duration" nur für die Zeilen, in denen das Ereignis "offer received" vorliegt
contacts_data.loc[contacts_data["event"] == "offer received", "duration"] = contacts_data.loc[contacts_data["event"] == "offer received", "duration"]

# Zeige die aktualisierten Daten an
contacts_data.info()



In [ ]:
merged_data

## Visualisierungen